In [1]:
import matplotlib_inline.backend_inline
from matplotlib import pyplot as plt
from astropy.table import Table
from astropy.io import fits
from pathlib import Path
import numpy as np
import eazy
import pickle
import gc
import os
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 25

WD = Path('/data1/hbahk/spherex-photoz/spherex-challenge/7ds_challenge')
os.chdir(WD)
TEMPDIR = WD.parent / 'BROWN_COSMOS'

from utils import plot_comp_hexbin

In [9]:
survey = 'IMS'
template = 'reddened'
zstep = '001'
catalog = '_sub'
extcorr = '_extcorrcat'

scheme = f'{template}_{zstep}_{survey}{catalog}{extcorr}'
print('Scheme:', scheme)

Scheme: reddened_001_IMS_sub_extcorrcat


In [10]:
def get_basecat_scheme(survey, template, zstep, catalog, extcorr, read_base=False):
    scheme = f'{template}_{zstep}_{survey}{catalog}{extcorr}'
    outdir = WD / 'output' / f'output_{scheme}'
    incatname = f'input_{survey}_subset.csv' if catalog == '_sub' else f'input_{survey}.csv'
    incatpath = WD / 'input' / incatname
    figdir = outdir / 'figures'

    figdir.mkdir(exist_ok=True, parents=True)

    if read_base:
        base = Table.read(outdir / 'result.fits')
    else:
        base = Table.read(incatpath)
        colnames = ['z_phot', 'z_phot_chi2', 'z160', 'z840', 'id']
        for label in colnames:
            base[label] = np.empty(len(base), dtype=float)

        if catalog == '_sub':
            outtab = Table.read(outdir / 'output.fits')
            for label in colnames:
                base[label] = outtab[label]
        else:
            for i in range(17):
                start_id = i*10000
                end_id = (i+1)*10000 if i < 16 else len(base)
                
                outtab = Table.read(outdir/f'output{i:02d}.fits')
                for label in colnames:
                    base[label][start_id:end_id] = outtab[label]
    
        base.write(outdir / 'result.fits', overwrite=True)
        
    return scheme, base, figdir

scheme, base, figdir = get_basecat_scheme(survey, template, zstep, catalog, extcorr)            

In [12]:
base

ID,HSC_i_MAG,z_psec,FLAG_ML,ra,dec,E_B-V_,Classic,EZzphot,F419,F420,F421,F422,F423,F424,F425,F426,F427,F428,F429,F430,F431,F432,F433,F434,F435,F436,F437,F438,E419,E420,E421,E422,E423,E424,E425,E426,E427,E428,E429,E430,E431,E432,E433,E434,E435,E436,E437,E438,z_phot,z_phot_chi2,z160,z840,id
int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,int64
738963,21.103617245014533,0.6216,0.0,150.37386852,2.5073432,0.017709000000000003,1149026,0.621499,2.6112547335475473,2.880827015647063,3.160543670939539,3.463803940982909,4.053598783827961,4.083686711306581,4.584210172368202,4.840942794035633,5.5103009678999815,7.041498401813331,8.252698095892232,9.369820125607335,9.905548836387496,11.198232063729364,12.000284219987202,11.928226634204313,14.883394175130066,14.092583818552985,14.444636249893238,16.073252786167917,0.13081352218667033,0.11817044949772083,0.11332117545091479,0.11422429288378155,0.11150009635951957,0.11870593904875958,0.1407258408619308,0.14070954157364457,0.1572242157892294,0.19155481187477152,0.189613925199004,0.20481156422405614,0.23172283068430174,0.30362998470967656,0.34035953083562187,0.4759623497362729,0.5301821000838206,0.5884940597372528,0.7420255058992078,1.0206752601973432,0.6291627,77.15479,0.6249,0.6566,738963
715464,24.377794723854887,2.8899,0.0,149.88668403,2.68432357,0.016769000000000003,1329160,2.897937,0.12151598926336958,0.3584347201955358,0.34733388829890327,0.5020661762557931,0.6328735585973589,0.5624087072335401,0.6242182920296966,0.6317265235773546,0.8703685142797455,0.6735055083475647,0.6820923757788929,0.6711097666622994,0.36135658886955546,0.7425879604192497,0.8094518172128994,0.080562110864082,1.0814976051164957,1.6565899729294236,0.4182443407641414,1.6563452546260182,0.1276735282700148,0.11386503800816561,0.10837556915229639,0.10821105381240785,0.10450832704019142,0.11137588868342073,0.13277876519430787,0.1319900871500606,0.14623050375575822,0.1780484914337715,0.16865507251109776,0.1791035677640766,0.20713651332271157,0.2796597971890184,0.3133472094030086,0.4544160323777001,0.5044138041437072,0.5660274983395606,0.7234725201698244,1.0043693898512953,2.791267,12.428036,0.2410,2.8741,715464
473138,24.860202039062116,0.9685,0.0,149.54777689,2.33306889,0.016961,963893,0.9487250000000002,0.04423105432110469,0.33244200770326515,0.19329279502659547,0.20700307966416848,0.33608156887004303,0.30427946112542836,0.4068177920792251,0.1730945521430573,0.3977236077172632,0.3610899384540743,0.18305546154165617,0.1214510551182146,0.32351521150066637,0.1134531475527269,0.4973176838466331,0.2944468259407356,0.7842863899305526,0.4253702374539035,1.3856680526005134,1.3285771976018221,0.12765764995177517,0.11381966964749346,0.10832531726999046,0.10811593976964756,0.10430687656325704,0.11116029353998198,0.1326074029694002,0.13177898328086543,0.14606151005445417,0.1778726131122731,0.16844405687958985,0.17889481282303707,0.20693622364411804,0.27955189350268445,0.3132328511825196,0.4543446087074313,0.5043764122691464,0.5659871649119609,0.7234518136522375,1.0043454126989328,3.2360702,8.038663,1.1693,3.1155,473138
413948,24.15871535721088,1.1851,0.0,149.61846183,1.75661489,0.017031,351738,1.171315,0.5501395346609569,0.6435289580567682,0.21845962725494047,0.36416562545202913,0.43631866454198637,0.32571096825483636,0.36727090636224763,0.700686729167902,0.7917810486927402,0.774690774706932,0.7296322530816454,0.6048745647892082,0.6610764034268706,0.7169923704213013,0.586646294988686,0.7452351801356245,0.4760404402437604,0.7421513595286009,0.37055251028952035,1.7200481265424403,0.12782117246321245,0.11396098320468745,0.10844739342794252,0.10823197921468611,0.1044345602

In [13]:
base.rename_column('z_psec', 'z_true')